In [2]:
import os
os.environ['CPLUS_INCLUDE_PATH']="/usr/include/gdal"
os.environ['C_INCLUDE_PATH']="/usr/include/gdal"

In [3]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:5 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease
Get:6 http://packages.cloud.google.com/apt cloud-sdk-bionic InRelease [6384 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [1230 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [73.6 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [1387 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [20.1 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [8815 B]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [59.3 kB]
Get

In [4]:
!apt-get --assume-yes install gdal-bin libgdal-dev python3-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  autotools-dev default-libmysqlclient-dev dh-python gdal-data
  gir1.2-harfbuzz-0.0 hdf5-helpers icu-devtools libaec-dev libaec0
  libarmadillo-dev libarmadillo8 libarpack2 libarpack2-dev libblas-dev
  libblas3 libboost-dev libboost1.65-dev libdap-dev libdap25 libdapclient6v5
  libdapserver7v5 libepsilon-dev libepsilon1 libexpat1-dev libfreexl-dev
  libfreexl1 libfyba-dev libfyba0 libgdal20 libgeos-3.6.2 libgeos-c1v5
  libgeos-dev libgeotiff-dev libgeotiff2 libgfortran4 libgif-dev
  libglib2.0-bin libglib2.0-dev libglib2.0-dev-bin libgraphite2-dev
  libharfbuzz-dev libharfbuzz-gobject0 libhdf4-0-alt libhdf4-alt-dev
  libhdf5-100 libhdf5-cpp-100 libhdf5-dev libicu-dev libicu-le-hb-dev
  libicu-le-hb0 libiculx60 libjbig-dev libjson-c-dev libjson-c3 libkml-dev
  libkmlbase1 libkmlconvenience1 libkmldom1 libkmlengine1 libkmlregionator1
  libk

In [5]:
!gdal-config --version

2.2.3


In [6]:
!pip install gdal==2.2.3

     |████████████████████████████████| 475 kB 8.0 MB/s eta 0:00:01
  Created wheel for gdal: filename=GDAL-2.2.3-cp37-cp37m-linux_x86_64.whl size=2168935 sha256=8e7e1c1f60537f8cc66ef371b17b804d5aa87ca7379933e22715bf979c2e2ccb
  Stored in directory: /root/.cache/pip/wheels/cc/bf/12/1946acb49f38536e147b1345b98bbfc4775802df1c0ec5b20f
Successfully built gdal


In [17]:
!python -c "from osgeo import gdal;print(gdal.__file__)"

/opt/conda/lib/python3.7/site-packages/osgeo/gdal.py


In [1]:
import sys, os

In [2]:
sys.path.append('/opt/conda/lib/python3.7/site-packages')

In [3]:
from osgeo import gdal

In [4]:
import re
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.io.gcp.gcsio import GcsIO
import tempfile

In [2]:
from getpass import getpass

In [ ]:
username='harrygibson'
password=getpass()
YEAR_FROM = 2019
YEAR_TO = 2020
DOY_START = 20
DOY_END = 40
TILE = '*'
BASE_URL = "http://e4ftl01.cr.usgs.gov"
platform = "MOLT"
product = "MOD11A2.006"
product_url = f"{BASE_URL}/{platform}/{product}"
product_url

In [6]:
def generate_selected_dates(year_from=2000, year_to=2020, doy_start=1, doy_end=-1):
    import calendar, time
    dates = []
    for year in range(year_from, year_to+1):
        if doy_end == -1:
            if calendar.isleap(year):
                end_day = 367
            else:
                end_day = 366
        else:
            end_day = doy_end
        dates_this_yr = [time.strftime("%Y.%m.%d", time.strptime("%d/%d" % (i, year),
                                                         "%j/%Y")) for i in
                 range(doy_start, end_day)]
        dates.extend(dates_this_yr)
    return dates

In [7]:
def get_existing_files(out_dir):
    # in case we need to do something different to list files on bucket
    return os.listdir(out_dir)

def load_page_text(url):
    import requests, time
    # nasa data pools are unavailable for maintenance on wednesday afternoons
    the_day_today = time.asctime().split()[0]
    the_hour_now = int(time.asctime().split()[3].split(":")[0])
    if the_day_today == "Wed" and 14 <= the_hour_now <= 17:
        LOG.info("Sleeping for %d hours... Yawn!" % (18 - the_hour_now))
        time.sleep(60 * 60 * (18 - the_hour_now))
    resp = requests.get(url)
    return resp.text
    
def parse_modis_dates (product_url, requested_dates, product, out_dir, check_existing_dates=False ):
    """Parse returned MODIS dates.

    This function gets the dates listing for a given MODIS products, and
    extracts the dates for when data is available. Further, it crosses these
    dates with the required dates that the user has selected and returns the
    intersection. Additionally, if the `checkExistingDates` flag is set, we'll check for
    files that might already be present in the system and skip them. Note
    that if a file failed in downloading, it might still be around
    incomplete.

    Parameters
    ----------
    url: str
        A top level product URL such as "http://e4ftl01.cr.usgs.gov/MOTA/MCD45A1.005/"
    dates: list
        A list of required dates in the format "YYYY.MM.DD"
    product: str
        The product name, MOD09GA.005
    out_dir: str
        The output dir
    checkExistingDates: bool
        Whether to check for present files
    Returns
    -------
    A (sorted) list with the dates that will be downloaded.
    """
    import time
    if check_existing_dates:
        product = product_url.strip('/').split('/')[-1]
        product_no_version = product.split(".")[0]
        already_here = fnmatch.filter(get_existing_files(out_dir),
                                      "%s*hdf" % product_no_version)
        already_here_dates = [x.split(".")[-5][1:]
                              for x in already_here]

    html = load_page_text(product_url).split('\n')

    available_dates = []
    for line in html:
        if line.find("href") >= 0 and \
                        line.find("[DIR]") >= 0:
            # Points to a directory
            the_date = line.split('href="')[1].split('"')[0].strip("/")
            if check_existing_dates:
                try:
                    modis_date = time.strftime("%Y%j",
                                               time.strptime(the_date,
                                                             "%Y.%m.%d"))
                except ValueError:
                    continue
                if modis_date in already_here_dates:
                    continue
                else:
                    available_dates.append(the_date)
            else:
                available_dates.append(the_date)

    dates = set(requested_dates)
    available_dates = set(available_dates)
    suitable_dates = list(dates.intersection(available_dates))
    suitable_dates.sort()
    return suitable_dates

In [8]:
real_dates = generate_selected_dates(YEAR_FROM, YEAR_TO, DOY_START, DOY_END)
dates_to_download = parse_modis_dates(product_url, real_dates, product, "C:\\temp")

In [9]:
dates_to_download

['2019.01.25', '2019.02.02', '2020.01.25', '2020.02.02']

In [10]:
p = beam.Pipeline(InteractiveRunner())

In [11]:
date_page_urls = (p | 'relevant_dates' >> beam.Create(dates_to_download)
                   | 'date_page_urls' >> beam.Map(lambda d: product_url + '/' + d)
                 )

In [12]:
ib.show(date_page_urls)

In [ ]:
all_2020_dates = generate_selected_dates(2020, 2020, 1, 366)
downloadable_for_2020 = parse_modis_dates(product_url, all_2020_dates, product, False)
p = beam.Pipeline(InteractiveRunner())
date_page_urls = (p | 'relevant_dates' >> beam.Create(downloadable_for_2020)
                 | 'date_page_urls' >> beam.Map(lambda d: product_url + '/' + d))

In [13]:
class GetUrlsForDate(beam.PTransform):
                
    def load_page_text_to_lines(self, url):
        import requests
        resp = requests.get(url)
        lines = resp.text.split('\n')
        return [(l, url) for l in lines]
        #return beam.Create(lines)
    
    def parse_hdf_from_line(self, textline, baseurl):
        if textline.find('.hdf"') != -1:
            return baseurl + '/' + textline.split('<a href="')[1].split('">')[0]
        
    def expand(self, pcoll):
        return (pcoll
                | "Load_page_lines" >> beam.FlatMap(self.load_page_text_to_lines)
                | "discover_hdf_urls" >> beam.MapTuple(lambda line, url:self.parse_hdf_from_line(line, url))
                | "remove_non_matching" >> beam.Filter(lambda l: l is not None)
               )
    

In [14]:
hdf_urls = (date_page_urls | GetUrlsForDate())

In [15]:
ib.show(hdf_urls)

In [16]:
bucketpath = "gs://hsg-dataflow-test/lst_download_dev"
hdf_bucket_path = os.path.join(bucketpath, 'hdf')


In [17]:
class check_existing_files(beam.PTransform):
    def __init__(self, hdf_bucket_path, req_tile_list="*"):
        gcs = GcsIO()
        self._existing = [os.path.basename(l) for l in list(gcs.list_prefix(hdf_bucket_path).keys())]
        self._required_tiles = req_tile_list
        
    def checktile(self, url):
        thistile = os.path.basename(url).split('.')[2]
        return self._required_tiles == "*" or thistile in self._required_tiles
    
    def expand(self, pcoll):
        stripped_existing = (pcoll | "remove_existing" >> beam.Filter(lambda l: os.path.basename(l) 
                                                        not in self._existing))
        if self._required_tiles == "*":
            return stripped_existing
        else:
            return stripped_existing | "remove_unrequired" >> beam.Filter(lambda l: self.checktile(l))
        

In [18]:
not_downloaded_yet = (hdf_urls | check_existing_files(hdf_bucket_path))

In [19]:
ib.show(not_downloaded_yet)

In [20]:
test_tiles = ['h17v03', 'h18v03', 'h17v04', 'h18v04']
need_to_download = (hdf_urls | check_existing_files(hdf_bucket_path, test_tiles))

In [21]:
ib.show(need_to_download)

In [22]:
class DownloadHdfsToBucket(beam.PTransform):
   
    def __init__(self, user, pw, hdf_bucket_path):
        import requests
        self._session = requests.Session()
        self._session.auth = (user, pw)
        self._hdf_bucket_path = hdf_bucket_path
        self._chunk_size = 8 * 1024 * 1024
    
    def download_file(self, url):
        import requests, tempfile, os
        req = self._session.request('get', url)
        resp = self._session.get(req.url, stream=True)
        product, datestr, fname = url.split('/')[-3:]
        bucketfilename = '/'.join([self._hdf_bucket_path, product, datestr, fname])
        gcs = GcsIO()
        with gcs.open(bucketfilename, 'w') as fp:
        #with open(tempfilename, 'wb') as fp:
            for chunk in resp.iter_content(chunk_size=self._chunk_size):
                if chunk:
                    fp.write(chunk)
            fp.flush()
            #os.fsync(fp)
        return("downloaded to "+bucketfilename)
    
    def expand(self, pcoll):
        return(pcoll | beam.Map(self.download_file))

In [23]:
download_results = need_to_download | DownloadHdfsToBucket(username, password, hdf_bucket_path)

In [24]:
ib.show(download_results)

In [25]:
class CreateVrtsForDays(beam.PTransform):
    def __init__(self, bucketpath):
        self._hdfpath = bucketpath
        gcs = GcsIO()
        self._existing = [l for l in list(gcs.list_prefix(hdf_bucket_path).keys())]
            
    def get_tilenames_for_day(self, day):
        return [f for f in self._existing if f.split('/')[-2] == day]
    
    def get_tmp_folder_for_day(self, day):
        tmpfolder = tempfile.gettempdir()
        workfolder = os.path.join(tmpfolder, day)
        return workfolder
    
    def localise_day_files(self, day):
        files = self.get_tilenames_for_day(day)
        tempfolder = self.get_tmp_folder_for_day(day)
        localpaths = []
        gcs = GcsIO()
        if not os.path.isdir(tempfolder):
            os.makedirs(tempfolder)
        for f in files:
            localname = os.path.join(tempfolder, os.path.basename(f))
            with gcs.open(f) as gcsfile, open(localname, 'wb') as localfile:
                localfile.write(gcsfile.read())
                localpaths.append(localname)
        return (day, localpaths)
    
    def build_lst_vrt_files(self,  day, paths):
        daytemplate = 'HDF4_EOS:EOS_GRID:"{}":MODIS_Grid_8Day_1km_LST:LST_Day_1km'
        nighttemplate = 'HDF4_EOS:EOS_GRID:"{}":MODIS_Grid_8Day_1km_LST:LST_Night_1km'
        daypaths = [daytemplate.format(f) for f in paths]
        nightpaths = [nighttemplate.format(f) for f in paths]
        thisfolder = os.path.dirname(paths[0])
        #day = os.path.basename(thisfolder)
        dayvrtfile = os.path.join(thisfolder, "LST_Day.{}.vrt".format(day))
        nightvrtfile = os.path.join(thisfolder, "LST_Night.{}.vrt".format(day))
        dayvrt = gdal.BuildVRT(dayvrtfile, daypaths)
        dayvrt.FlushCache()
        dayvrt = None
        nightvrt = gdal.BuildVRT(nightvrtfile, nightpaths)
        nightvrt.FlushCache()
        nightvrt = None     
        return (dayvrtfile, nightvrtfile)
        
    def expand(self, pcoll):
        return (pcoll |  beam.Map(self.localise_day_files)
             | "build_day_night_vrts" >> beam.MapTuple(lambda d, p: self.build_lst_vrt_files(d,p)) 
             | "flat_list_of_vrts" >> beam.FlatMap(lambda t: t)
            )
         # groups by day

In [26]:
p = beam.Pipeline(InteractiveRunner())
datesPipeline = p | "dates for processing " >> beam.Create(dates_to_download)

In [27]:
ib.show(datesPipeline)

In [28]:
vrts = datesPipeline | CreateVrtsForDays(hdf_bucket_path)

In [29]:
ib.show(vrts )

In [30]:
import numexpr as ne

In [35]:
class TranslateVrtToLstTiff(beam.PTransform):
    
    def get_out_name(self, vrtname):
        return vrtname.replace('.vrt', '.sinusoidal.tif')
    
    def expand(self, pColl):
        lst_calc = "band_data * 0.02 + (-273.15)"
        return pColl | beam.Map(lambda v: run_singleband_calculation(v, self.get_out_name(v), lst_calc))
        

In [36]:
tiffs = vrts | TranslateVrtToLstTiff()

In [37]:
ib.show(tiffs)

In [75]:
class create_projected_tiffs(beam.PTransform):
    
    def __init__(self, ForceGlobalExtent = False):
        self._forceglobal = ForceGlobalExtent
        
    def warpfile(self, sinusFile):
        cOpts = ["TILED=YES", "BIGTIFF=YES", "COMPRESS=LZW", "NUM_THREADS=ALL_CPUS"]
        if self._forceglobal:
            wo = gdal.WarpOptions(format='GTiff', 
                          outputBounds=[-180, -90, 180, 90], 
                          xRes=1/120.0, yRes=-1/120.0, dstSRS='EPSG:4326',
                          creationOptions=cOpts, multithread=True, dstNodata=-9999, warpMemoryLimit=2048)
        else:
            wo = gdal.WarpOptions(format='GTiff', 
                          xRes=1/120.0, yRes=-1/120.0, dstSRS='EPSG:4326',
                          targetAlignedPixels="YES",
                          creationOptions=cOpts, multithread=True, dstNodata=-9999, warpMemoryLimit=2048)
            
        outname = sinusFile.replace('.sinusoidal', '')
        gdal.Warp(outname, sinusFile, options=wo)
        return outname
    
    def expand(self, pColl):
        return pColl | beam.Map(self.warpfile)


In [76]:
warped = tiffs | create_projected_tiffs()

In [77]:
ib.show(warped)

In [ ]:
#africa: h16_23 v5_12
import itertools
tiles = [f'h{pair[0]:02}v{pair[1]:02}' for pair in (itertools.product(range(16,24), range(5,13)))]

all_2020_dates = generate_selected_dates(2020, 2020, 1, 366)
downloadable_for_2020 = parse_modis_dates(product_url, all_2020_dates, product, False)
p = beam.Pipeline(InteractiveRunner())
date_page_urls = (p | 'relevant_dates' >> beam.Create(downloadable_for_2020)
                 | 'date_page_urls' >> beam.Map(lambda d: product_url + '/' + d))
hdf_urls = date_page_urls | GetUrlsForDate()
required_for_download = hdf_urls | check_existing_files(hdf_bucket_path, tiles)

download_results = need_to_download | DownloadHdfsToBucket(username, passwork, hdf_bucket_path)

mosaicPipeline = beam.Pipeline(InteractiveRunner()) | "dates for processing " >> beam.Create(dates_to_download)
vrts = mosaicPipeline | CreateVrtsForDay(hdf_bucket_path)
tiffs = vrts | TranslateVrtToLstTiff() | create_projected_tiffs() | beam.ParDo(cleanup())

In [79]:
hdf_bucket_path

'gs://hsg-dataflow-test/lst_download_dev/hdf'

In [81]:
day_bucket_path = bucketpath + '/' + "output/lst_day"
night_bucket_path = bucketpath + '/' + "output/lst_night"

In [106]:
class cleanup(beam.DoFn):
   
    def process(self, finaltif):
        date = os.path.basename(os.path.dirname(finaltif))
        parts = os.path.basename(finaltif).split('.')
        outname = parts[0] + "_Unfilled." + parts[1] + "." + parts[2]+parts[3]+".Data.1km.Data.tif"
        if parts[0].find("Day")>0:
            gsPath = day_bucket_path + "/" + outname
        elif parts[0].find("Night")>0:
            gsPath = night_bucket_path + "/" + outname
        else:
            return
        gcs = GcsIO()
        with gcs.open(gsPath, 'w') as gcsfile, open(finaltif, 'rb') as localfile:
            gcsfile.write(localfile.read())
        os.remove(finaltif)
        yield gsPath
        

In [107]:
uploaded = warped | beam.ParDo(cleanup())

In [108]:
ib.show(uploaded)

In [34]:
def run_singleband_calculation(input_singleband_file, out_file, calc, out_type='Float32'):
    '''Calc must be the calculation to apply to the data from input_singleband_file, specified as a string which will be 
    eval'd against the data which will exist in a variable called band_data. i.e. to specify doubling the data then subtracting 
    three, provide calc="(band_data * 2.0) - 3.0"'''
    #input_datasets = []
    #myBands = []
    #myDataType = []
    #myDataTypeNum = []
    #myNDV = []
    DimensionsCheck = None
    
    ds = gdal.Open(input_singleband_file, gdal.GA_ReadOnly)
    if not ds:
        raise IOError("Error opening input file {}".format(input_file))
    input_dataset = ds
    inputDataType = (gdal.GetDataTypeName(ds.GetRasterBand(1).DataType))
    inputDataTypeNum = (ds.GetRasterBand(1).DataType)
    inputNDV = (ds.GetRasterBand(1).GetNoDataValue())
    
    DimensionsCheck = [ds.RasterXSize, ds.RasterYSize]

    if os.path.isfile(out_file):
        os.remove(out_file)
    # gdal_calc does this but it isn't valid as two int datasets can result in a float!
    # outType = gdal.GetDataTypeName(max(myDataTypeNum))
    
    #create the output file
    outDriver = gdal.GetDriverByName("GTiff")
    cOpts =  ["TILED=YES", "SPARSE_OK=TRUE", "BLOCKXSIZE=1024", "BLOCKYSIZE=1024", "BIGTIFF=YES", "COMPRESS=LZW", "NUM_THREADS=ALL_CPUS"]
    outDS = outDriver.Create(out_file, DimensionsCheck[0], DimensionsCheck[1], 1, gdal.GetDataTypeByName(out_type), cOpts)
    outDS.SetGeoTransform(input_dataset.GetGeoTransform())
    outDS.SetProjection(input_dataset.GetProjection())
    DefaultNDVLookup = {'Byte': 255, 'UInt16': 65535, 'Int16': -32767, 'UInt32': 4294967293, 'Int32': -2147483647, 'Float32': 3.402823466E+38, 'Float64': 1.7976931348623158E+308}
    outBand = outDS.GetRasterBand(1)
    outNDV = DefaultNDVLookup[out_type]
    outBand.SetNoDataValue(outNDV)
    outBand = None
    
    # vrt file reports a block size of 128*128 but the underlying hdf block size is 1200*100
    # so hard code this or some clean multiple : this minimises disk access
    myBlockSize = [4800,4800]
    nXValid = myBlockSize[0]
    nYValid = myBlockSize[1]
    nXBlocks = (int)((DimensionsCheck[0] + myBlockSize[0] - 1) / myBlockSize[0]);
    nYBlocks = (int)((DimensionsCheck[1] + myBlockSize[1] - 1) / myBlockSize[1]);
    
    for x in range(0, nXBlocks):
        if x == nXBlocks-1:
            nXValid = DimensionsCheck[0] - x * myBlockSize[0]
        
        myX = x * myBlockSize[0]
        
        nYValid = myBlockSize[1]
        myBufSize = nXValid * nYValid
        
        for y in range(0, nYBlocks):
            if y == nYBlocks-1:
                nYValid = DimensionsCheck[1] - y * myBlockSize[1]
                myBufSize = nXValid * nYValid
                
            myY = y * myBlockSize[1]
            band_data = input_dataset.GetRasterBand(1).ReadAsArray(xoff=myX, yoff=myY, 
                                                                       win_xsize=nXValid, win_ysize=nYValid)
            nodata_locs = band_data == inputNDV
            
            try:
                result = ne.evaluate(calc)
            except:
                raise
            
            # apply ndv (set nodata cells to zero then add nodata value to these cells)
            result = ((1 * (nodata_locs==0))*result + (outNDV * nodata_locs))
            
            outBand = outDS.GetRasterBand(1)
            outBand.WriteArray(result, xoff=myX, yoff=myY)
    return out_file

In [79]:
def run_multiband_calculation(input_singleband_files, outfile, calc):
    input_datasets = []
    myBands = []
    myDataType = []
    myDataTypeNum = []
    myNDV = []
    DimensionsCheck = None
    
    for input_file in input_singleband_file:
        ds = gdal.Open(input_file, gdal.GA_ReadOnly)
        if not ds:
            raise IOError("Error opening input file {}".format(input_file))
        input_datasets.append(ds)
        myDataType.append(gdal.GetDataTypeName(ds.GetRasterBand(1).DataType))
        myDataTypeNum.append(ds.GetRasterBand(1).DataType)
        myNDV.append(ds.GetRasterBand(1).GetNoDataValue())
        if DimensionsCheck:
            if DimensionsCheck != [ds.RasterXSize, ds.RasterYSize]:
                raise Exception("Error! Dimensions of file %s (%i, %i) are different from other files (%i, %i).  Cannot proceed" %
                                    (input_file, ds.RasterXSize, ds.RasterYSize, DimensionsCheck[0], DimensionsCheck[1]))
        else:
            DimensionsCheck = [ds.RasterXSize, ds.RasterYSize]
    
    if os.path.isfile(outfile):
        os.remove(outfile)
    outType = gdal.GetDataTypeName(max(myDataTypeNum))
    outDriver = gdal.GetDriverByName("GTiff")
    outDS = outDriver.Create(outfile, DimensionsCheck[0], DimensionsCheck[1], 1, gdal.GetDataTypeByName('Float32'), ["TILED=YES" "SPARSE_OK=TRUE" "BLOCKXSIZE=1024" "BLOCKYSIZE=1024"])
    outDS.SetGeoTransform(input_datasets[0].GetGeoTransform())
    outDS.SetProjection(input_datasets[0].GetProjection())
    DefaultNDVLookup = {'Byte': 255, 'UInt16': 65535, 'Int16': -32767, 'UInt32': 4294967293, 'Int32': -2147483647, 'Float32': 3.402823466E+38, 'Float64': 1.7976931348623158E+308}
    outBand = outDS.GetRasterBand(1)
    outNDV = DefaultNDVLookup[outType]
    outBand.SetNoDataValue(outNDV)
    outBand = None
    
    # vrt file reports a block size of 128*128 but the underlying hdf block size is 1200*100
    # so hard code this or some clean multiple : this minimises disk access
    myBlockSize = [4800,4800]
    nXValid = myBlockSize[0]
    nYValid = myBlockSize[1]
    nXBlocks = (int)((DimensionsCheck[0] + myBlockSize[0] - 1) / myBlockSize[0]);
    nYBlocks = (int)((DimensionsCheck[1] + myBlockSize[1] - 1) / myBlockSize[1]);
    
    for x in range(0, nXBlocks):
        if x == nXBlocks-1:
            nXValid = DimensionsCheck[0] - x * myBlockSize[0]
        
        myX = x * myBlockSize[0]
        
        nYValid = myBlockSize[1]
        myBufSize = nXValid * nYValid
        
        for y in range(0, nYBlocks):
            if y == nYBlocks-1:
                nYValid = DimensionsCheck[1] - y * myBlockSize[1]
                myBufSize = nXValid * nYValid
                
            myY = y * myBlockSize[1]
            band_data = input_datasets[0].GetRasterBand(1).ReadAsArray(xoff=myX, yoff=myY, 
                                                                       win_xsize=nXValid, win_ysize=nYValid)
            nodata_locs = band_data == myNDV[0]
            
            try:
                result = ne.evaluate(calc)
            except:
                raise
            
            # apply ndv (set nodata cells to zero then add nodata value to these cells)
            result = ((1 * (nodata_locs==0))*result + (outNDV * nodata_locs))
            
            outBand = outDS.GetRasterBand(1)
            outBand.WriteArray(result, xoff=myX, yoff=myY)

In [91]:
lst_calc = "band_data * 0.02 + (-273.15)"
run_singleband_calculation("/tmp/2020.02.02/LST_Day.2020.02.02.vrt", "/tmp/2020.02.02/testday.tif", lst_calc)